# Importing necessary modules

In [1]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from multiprocessing import Pool
from collections import Counter

# Text Pre-processing Functions

In [2]:
contraction_mapping = {"that'll":"that will","ain’t": "is not", "aren’t": "are not","can’t": "cannot", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
                          "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hasn’t": "has not", "haven’t": "have not",
                          "he’d": "he would","he’ll": "he will", "he’s": "he is", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is",
                          "I’d": "I would", "I’d’ve": "I would have", "I’ll": "I will", "I’ll’ve": "I will have","I’m": "I am", "I’ve": "I have", "i’d": "i would",
                          "i’d’ve": "i would have", "i’ll": "i will",  "i’ll’ve": "i will have","i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
                          "it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have","it’s": "it is", "let’s": "let us", "ma’am": "madam",
                          "mayn’t": "may not", "might’ve": "might have","mightn’t": "might not","mightn’t’ve": "might not have", "must’ve": "must have",
                          "mustn’t": "must not", "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have","o’clock": "of the clock",
                          "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have",
                          "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "she’s": "she is",
                          "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have","so’s": "so as",
                          "this’s": "this is","that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
                          "there’d’ve": "there would have", "there’s": "there is", "here’s": "here is","they’d": "they would", "they’d’ve": "they would have",
                          "they’ll": "they will", "they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have",
                          "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are",
                          "we’ve": "we have", "weren’t": "were not", "what’ll": "what will", "what’ll’ve": "what will have", "what’re": "what are",
                          "what’s": "what is", "what’ve": "what have", "when’s": "when is", "when’ve": "when have", "where’d": "where did", "where’s": "where is",
                          "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have", "who’s": "who is", "who’ve": "who have",
                          "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
                          "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all",
                          "y’all’d": "you all would","y’all’d’ve": "you all would have","y’all’re": "you all are","y’all’ve": "you all have",
                          "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
                          "you’re": "you are", "you’ve": "you have","ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                          "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                          "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                          "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                          "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                          "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                          "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                          "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                          "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                          "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                          "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                          "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                          "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                          "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                          "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                          "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                          "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                          "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                          "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                          "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                          "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                          "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                          "you're": "you are", "you've": "you have","n't":'not'}



def data_preprocess_english(sentence):
    """ data_preprocess_english() function takes a list of english sentences as input
        and returns the pre-processed english sentences list
        Input : ['#Spotlight Take Me To Paradise by Arsonist MC #WNIAGospel http://t.co/1he4UfaWZm @arsonistmusic http://t.co/BNhtxAEZMM']
        Output : ['spotlight take paradise arsonist wniagospel']
    """
    clean_data=[]

    # Iterating through each tweet
    for line in tqdm(sentence):
        x = line.lower()              #lower-casing the words
        word_list = []                

        # Iterating through each word in the tweet
        for word in x.split():      # contraction mapping
            if word in contraction_mapping:
                #print(word)
                word_list.append(contraction_mapping[word])
            else:
                word_list.append(word)

        x = ' '.join(word_list)                    # converting words list into single string
        x = re.sub("'s",' ',x)                     # removing 's, Example : it's --> it
        x = re.sub('\n',' ',x)                     # removing newline character
        x = re.sub('[6वडई8मरग@&ै।भ4ू+/:3टख#ी],स-ाएइ7–हु!.0यद्क़1े\'ल9थबऔआ\\शॉउंतजफ5?[पि$न2"ोचघ]',' ', x)
        x = re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~|।]',' ',x)  # removing punctuations
        x = re.sub('[^a-z]',' ',x)                 # removing non-alphabetic characters
        x = re.sub('\s+',' ',x)                    # replacing multiple spaces with single spaces
        x = x.strip()                              # Removing leftmost and rightmost spaces
        clean_data.append(x)

    return clean_data


def data_preprocess_hindi(sentence):
    """ data_preprocess_hindi() function takes a list of hindi sentences as input
        and returns the pre-processed hindi sentences list
        Input : ['whtie माइक्रोवेव ओवन।']
        Output : ['माइक्रोवेव ओवन']
    """
    clean_data = []

    # Iterating through each tweet
    for line in tqdm(sentence):
        x = re.sub('\n',' ',line)                                          # removing newline character
        x = re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~|।०-९ ॥ॽ]',' ',x)   # removing punctuations
        x = re.sub("[K१[२zs#\u200boXWT8P+४A|r2mNwpHdl4!३kn\\'F$5C-u९v63_/ZU1yBO\\0I7cD?9M.e@xa६]gfbV८hL’\u200dtSRi:EqG।,Y]",' ',x)
        x = re.sub('[०-९।॥ॽ]',' ',x)
        x = re.sub('[^\u0900-\u097F]',' ',x)
        x = re.sub('\s+',' ',x)                            # replacing multiple spaces with single spaces
        x = x.strip()                                      # Removing leftmost and rightmost spaces
        clean_data.append(x)

    return clean_data

# Reading Hindi Visual Genome Dataset

In [3]:
df = pd.read_csv(os.path.join('./Dataset/text','HVG.csv'))
print(df.columns)
df

Index(['image_id', 'X', 'Y', 'Width', 'Height', 'English Text', 'Hindi Text',
       'P_TYPE'],
      dtype='object')


,image_id,X,Y,Width,Height,English Text,Hindi Text,P_TYPE
0,11,45,30,545.0,436,it is an indoor scene,यह एक इनडोर दृश्य है,train
1,17,306,239,189.0,94,Computer screens turned on,कंप्यूटर स्क्रीन चालू,train
2,18,664,241,93.0,126,man has short hair,आदमी के छोटे बाल हैं,train
3,21,1,113,120.0,138,photo album open on an adult's lap,एक वयस्क की गोद में फोटो एल्बम खुला,train
4,23,105,178,148.0,209,there is a group of girls beside the black car,काली कार के पास लड़कियों का एक समूह है,train
...,...,...,...,...,...,...,...,...
32918,2358720,77,20,35.0,52,a fence between the field and players,मैदान और खिलाड़ियों के बीच एक बाड़,challenge
32919,2414143,445,110,52.0,76,Sponsor signs hanging on the walls of the tenn...,टेनिस कोर्ट की दीवारों पर प्रायोजक के निशान लट...,challenge
32920,2941,395,287,137.0,153,Many stairs up to a second story.,कई सीढ़ियाँ एक दूसरी कहानी तक।,challenge
32921,2334985,235,184,244.0,82,section of fine gray sand,महीन भूरे बालू का खंड,challenge


# Pre-processing Text 

In [4]:
df['English Text Pre'] = data_preprocess_english(df['English Text'])
df['Hindi Text Pre'] = data_preprocess_hindi(df['Hindi Text'])
df = df[['image_id', 'X', 'Y', 'Width', 'Height','P_TYPE','English Text','English Text Pre', 'Hindi Text','Hindi Text Pre']]
df

100%|█████████████████████████████████| 32923/32923 [00:00<00:00, 288405.98it/s]


,image_id,X,Y,Width,Height,P_TYPE,English Text,English Text Pre,Hindi Text,Hindi Text Pre
0,11,45,30,545.0,436,train,it is an indoor scene,it is an indoor scene,यह एक इनडोर दृश्य है,यह एक इनडोर दृश्य है
1,17,306,239,189.0,94,train,Computer screens turned on,computer screens turned on,कंप्यूटर स्क्रीन चालू,कंप्यूटर स्क्रीन चालू
2,18,664,241,93.0,126,train,man has short hair,man has short hair,आदमी के छोटे बाल हैं,आदमी के छोटे बाल हैं
3,21,1,113,120.0,138,train,photo album open on an adult's lap,photo album open on an adult lap,एक वयस्क की गोद में फोटो एल्बम खुला,एक वयस्क की गोद में फोटो एल्बम खुला
4,23,105,178,148.0,209,train,there is a group of girls beside the black car,there is a group of girls beside the black car,काली कार के पास लड़कियों का एक समूह है,काली कार के पास लड़कियों का एक समूह है
...,...,...,...,...,...,...,...,...,...,...
32918,2358720,77,20,35.0,52,challenge,a fence between the field and players,a fence between the field and players,मैदान और खिलाड़ियों के बीच एक बाड़,मैदान और खिलाड़ियों के बीच एक बाड़
32919,2414143,445,110,52.0,76,challenge,Sponsor signs hanging on the walls of the tenn...,sponsor signs hanging on the walls of the tenn...,टेनिस कोर्ट की दीवारों पर प्रायोजक के निशान लट...,टेनिस कोर्ट की दीवारों पर प्रायोजक के निशान लट...
32920,2941,395,287,137.0,153,challenge,Many stairs up to a second story.,many stairs up to a second story,कई सीढ़ियाँ एक दूसरी कहानी तक।,कई सीढ़ियाँ एक दूसरी कहानी तक
32921,2334985,235,184,244.0,82,challenge,section of fine gray sand,section of fine gray sand,महीन भूरे बालू का खंड,महीन भूरे बालू का खंड


# Dataset Details

In [5]:
print("Total rows including all partitions : ",len(df))
print("Partition wise count : ",dict(df['P_TYPE'].value_counts()))

Total rows including all partitions :  32923
Partition wise count :  {'train': 28930, 'test': 1595, 'challenge': 1400, 'val': 998}


## Removing Empty Sentences (English)

In [6]:
flag = []
row = 0
for i in df['English Text Pre']:
    if len(i.split()) == 0:
        print(row,i)
        flag.append(False)
    else:
        flag.append(True)
    row += 1
    
df = df[flag]
df

4436 
6328 
9368 
9562 


,image_id,X,Y,Width,Height,P_TYPE,English Text,English Text Pre,Hindi Text,Hindi Text Pre
0,11,45,30,545.0,436,train,it is an indoor scene,it is an indoor scene,यह एक इनडोर दृश्य है,यह एक इनडोर दृश्य है
1,17,306,239,189.0,94,train,Computer screens turned on,computer screens turned on,कंप्यूटर स्क्रीन चालू,कंप्यूटर स्क्रीन चालू
2,18,664,241,93.0,126,train,man has short hair,man has short hair,आदमी के छोटे बाल हैं,आदमी के छोटे बाल हैं
3,21,1,113,120.0,138,train,photo album open on an adult's lap,photo album open on an adult lap,एक वयस्क की गोद में फोटो एल्बम खुला,एक वयस्क की गोद में फोटो एल्बम खुला
4,23,105,178,148.0,209,train,there is a group of girls beside the black car,there is a group of girls beside the black car,काली कार के पास लड़कियों का एक समूह है,काली कार के पास लड़कियों का एक समूह है
...,...,...,...,...,...,...,...,...,...,...
32918,2358720,77,20,35.0,52,challenge,a fence between the field and players,a fence between the field and players,मैदान और खिलाड़ियों के बीच एक बाड़,मैदान और खिलाड़ियों के बीच एक बाड़
32919,2414143,445,110,52.0,76,challenge,Sponsor signs hanging on the walls of the tenn...,sponsor signs hanging on the walls of the tenn...,टेनिस कोर्ट की दीवारों पर प्रायोजक के निशान लट...,टेनिस कोर्ट की दीवारों पर प्रायोजक के निशान लट...
32920,2941,395,287,137.0,153,challenge,Many stairs up to a second story.,many stairs up to a second story,कई सीढ़ियाँ एक दूसरी कहानी तक।,कई सीढ़ियाँ एक दूसरी कहानी तक
32921,2334985,235,184,244.0,82,challenge,section of fine gray sand,section of fine gray sand,महीन भूरे बालू का खंड,महीन भूरे बालू का खंड


## Removing Empty Sentences (Hindi)


In [7]:
flag = []
row = 0
for i in df['Hindi Text Pre']:
    if len(i.split()) == 0:
        print(row,i)
        flag.append(False)
    else:
        flag.append(True)
    row += 1
    
df = df[flag]
df

3122 
14461 
14655 
28721 


,image_id,X,Y,Width,Height,P_TYPE,English Text,English Text Pre,Hindi Text,Hindi Text Pre
0,11,45,30,545.0,436,train,it is an indoor scene,it is an indoor scene,यह एक इनडोर दृश्य है,यह एक इनडोर दृश्य है
1,17,306,239,189.0,94,train,Computer screens turned on,computer screens turned on,कंप्यूटर स्क्रीन चालू,कंप्यूटर स्क्रीन चालू
2,18,664,241,93.0,126,train,man has short hair,man has short hair,आदमी के छोटे बाल हैं,आदमी के छोटे बाल हैं
3,21,1,113,120.0,138,train,photo album open on an adult's lap,photo album open on an adult lap,एक वयस्क की गोद में फोटो एल्बम खुला,एक वयस्क की गोद में फोटो एल्बम खुला
4,23,105,178,148.0,209,train,there is a group of girls beside the black car,there is a group of girls beside the black car,काली कार के पास लड़कियों का एक समूह है,काली कार के पास लड़कियों का एक समूह है
...,...,...,...,...,...,...,...,...,...,...
32918,2358720,77,20,35.0,52,challenge,a fence between the field and players,a fence between the field and players,मैदान और खिलाड़ियों के बीच एक बाड़,मैदान और खिलाड़ियों के बीच एक बाड़
32919,2414143,445,110,52.0,76,challenge,Sponsor signs hanging on the walls of the tenn...,sponsor signs hanging on the walls of the tenn...,टेनिस कोर्ट की दीवारों पर प्रायोजक के निशान लट...,टेनिस कोर्ट की दीवारों पर प्रायोजक के निशान लट...
32920,2941,395,287,137.0,153,challenge,Many stairs up to a second story.,many stairs up to a second story,कई सीढ़ियाँ एक दूसरी कहानी तक।,कई सीढ़ियाँ एक दूसरी कहानी तक
32921,2334985,235,184,244.0,82,challenge,section of fine gray sand,section of fine gray sand,महीन भूरे बालू का खंड,महीन भूरे बालू का खंड


In [8]:
print("After Removing Empty Sentences")

print("Total rows including all partitions : ",len(df))
print("Partition wise count : ",dict(df['P_TYPE'].value_counts()))

After Removing Empty Sentences
Total rows including all partitions :  32915
Partition wise count :  {'train': 28922, 'test': 1595, 'challenge': 1400, 'val': 998}


# Removing Duplicate sentences partition wise

In [9]:
df_train = df[df['P_TYPE'] == 'train']
df_train = df_train.drop_duplicates()
df_val = df[df['P_TYPE'] == 'val']
df_val = df_val.drop_duplicates()
df_test = df[df['P_TYPE'] == 'test']
df_test = df_test.drop_duplicates()
df_challenge = df[df['P_TYPE'] == 'challenge']
df_challenge = df_challenge.drop_duplicates()

df = pd.concat([df_train, df_val, df_test, df_challenge], ignore_index = True)    # Concatenate all partitions

In [10]:
print("After Removing Duplicate Sentences partition wise")

print("Total rows including all partitions : ",len(df))
print("Partition wise count : ",dict(df['P_TYPE'].value_counts()))

After Removing Duplicate Sentences partition wise
Total rows including all partitions :  32912
Partition wise count :  {'train': 28919, 'test': 1595, 'challenge': 1400, 'val': 998}


### Saving Dataset in CSV file

In [11]:
df.to_csv(os.path.join('./Dataset/text',"HVG_Pre_Processed.csv"), index = None)